In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import geopandas as gpd
from shapely.geometry import Polygon
import os
import pyproj

In [ ]:
site_id = "01507000"
year = 2000

In [ ]:
# basin shapefile
gdf = gpd.read_file("%s.basin_bound.gpkg"%site_id)
gdf.crs = "EPSG:4326"

In [ ]:
# domain file
domain_ds = xr.open_dataset("namerica_domain.%s.nc"%site_id)

In [ ]:
# meteorological forcing data
met_ds = xr.open_dataset(f"ERA5.{site_id}.regrid.{year}.nc")

In [ ]:
met_ds

### Units for Each Variable can be seen by clicking the page symbol next to the three pie symbol

# 0. Visualize the dataset!

In [ ]:
proj1 = ccrs.Mercator()

### 0.1. Let's first take a look at the targeted basin

In [ ]:
plt.figure(dpi=150)
ax = plt.axes(projection=proj1)
gdf.to_crs(pyproj.CRS(proj1.proj4_params)).plot(ax=ax,facecolor='none',edgecolor='orange',zorder=3)


### 0.2. Add our model domain into the map and make sure that our domain covers the entire river basin!

In [ ]:
plt.figure(dpi=150)
ax = plt.axes(projection=proj1)
domain_ds.mask.plot(x='lon',y='lat',transform=ccrs.PlateCarree(),add_colorbar=False,
                    facecolor='none',edgecolor='silver', zorder=2,lw=0.4)
gdf.to_crs(pyproj.CRS(proj1.proj4_params)).plot(ax=ax,facecolor='none',edgecolor='orange',zorder=3)


### 0.3. Add the met forcing into the map and make sure that every grid cells in the domain have forcing data!

In [ ]:
plt.figure(dpi=150)
ax = plt.axes(projection=proj1)
(met_ds.MTPR.mean(dim='time')*366*86400).plot(x='lon',y='lat',transform=ccrs.PlateCarree(),
                                  cmap='Blues',edgecolor='k',zorder=1,lw=0.4)
domain_ds.mask.plot(x='lon',y='lat',transform=ccrs.PlateCarree(),add_colorbar=False,
                    facecolor='none',edgecolor='silver', zorder=2,lw=0.4)
gdf.to_crs(pyproj.CRS(proj1.proj4_params)).plot(ax=ax,facecolor='none',edgecolor='orange',zorder=3)


### After checking the map above, we find that the spatial resolution of the met forcing is coarser than that of our domain. Therefore, we will need to perform regridding to the met forcing data. We will take care of it later!

# Targeted variables and units for VIC
```
Variable		Description						        Units
AIR_TEMP		Average air temperature					C
PREC			Total precipitation (rain and snow)		mm
PRESSURE		Atmospheric pressure					kPa
SWDOWN			Incoming shortwave radiation			W/m2
LWDOWN			Incoming longwave radiation				W/m2
VP				Vapor pressure							kPa
WIND			Wind speed								m/s
```

# Step 1: Find what variables are available in the ERA5 data and how we can convert it to the VIC input

[Variables in VIC]: [Variables available in ERA5]
```
AIR_TEMP: VAR_2T
PREC: MTPR
PRESSURE: SP
SWDOWN: MSDWSWRF
LWDOWN: MSDWLWRF
VP: VAR_2D (how to calculate vapor pressure from dew point temperature: https://www.weather.gov/media/epz/wxcalc/vaporPressure.pdf)
WIND:  VAR_10U, VAR_10V (WIND = np.sqrt(VAR_10U**2 + VAR_10V**2))
```

In [ ]:
# Insert the code to calculate wind speed using the 2D wind variables (VAR_10U, VAR_10V)
WIND = np.sqrt(met_ds.VAR_10U**2 + met_ds.VAR_10V**2)

In [ ]:
WIND

## How to calculate vapor pressure
VP: VAR_2D (how to calculate vapor pressure from dew point temperature)

https://www.weather.gov/media/epz/wxcalc/vaporPressure.pdf

## How can we check the units for `VAR_2D`?

In [ ]:
met_ds.VAR_2D.attrs['units']

In [ ]:
# Note that unit for the dew point temperature is provided in Klevin, we need to 
# convert it to Celsius.
VAR_2D_C = met_ds.VAR_2D - 273.15

In [ ]:
# Insert the code to calculate vapor pressure (VP) using dew point
VP = 

# Step 2: Are the units in ERA5 variables the same as the units in VIC input data?

## How can we check the units for the Surface Pressure (SP)?

In [ ]:
met_ds.SP.attrs['units']

#### We especially need to watch out for the following variables.
1. SP: surface pressure. The unit for SP is Pa in ERA5. VIC requires kPa. So we need to divide SP(in Pa) by 1000 to get SP (in kPa). 
2. VAR_2T: air temperature. The unit for VAR_2T is K in ERA5. VIC requires C. So we need to subtract VAR_2T (in K) by 273.15 to get VAR_2T in Celsuis. 
3. MTPR: precipitation. The unit for MTPR is kg m**-2 s**-1 while VIC requires mm/timestep. 

$$
  \frac{kg}{m^2 \cdot s} = \frac{kg}{m^2 \cdot s} \cdot  \frac{1 m^3}{1000 kg} \cdot \frac{1000mm}{1m} = \frac{mm}{s}
$$

    And our timestep is $1h = 3600s$. So we need to multiply MTPR by 3600, then we can get our targeted variable

4. VP: the calculated vapor pressure from the equation above is millibars (mb) or hectoPascals (hPa). VIC requires kPa. So we only need to multiply VP(hPa) by 0.1 to get VP in kPa.

In [ ]:
# Insert your code here to convert the units for SP
SP_kPa = met_ds.SP/1000

In [ ]:
# Insert your code here to convert the units for VAR_2T
VAR_2T_C = 

In [ ]:
# Insert your code here to convert the units of MRPT
MTPR_mm = 

In [ ]:
# Insert your code here to convert the units of VP
VP_kPa = 

### It is important to add the units attributes back to those variables

In [ ]:
SP_kPa.attrs['units'] = 'kPa'
VAR_2T_C.attrs['units'] = 'deg C'
MTPR_mm.attrs['units'] = 'mm/timestep'
VP_kPa.attrs['units'] = 'kPa'
WIND.attrs['units'] = 'm s**-1'

# Step 3: Update the dataarray in your dataset using correct variables in correct units

In [ ]:
ds_met_data = xr.Dataset({'SP':SP_kPa,
                          'VAR_2T':VAR_2T_C,
                          'MTPR':MTPR_mm,
                          'VP':VP_kPa,
                          'WIND':WIND,
                          'MSDWLWRF':met_ds.MSDWLWRF,
                          'MSDWSWRF':met_ds.MSDWSWRF})

In [ ]:
ds_met_data.to_netcdf("ERA5.%s.orig_rsln.2012.nc"%site_id)

# Step 4: Regrid the met forcings to the targeted domain grids!
I already took care of this for you. It is just impossible to install xESMF package properly in the virtual environment.